In [ ]:
import logging
from pprint import pformat
from collections import namedtuple
import random
from copy import deepcopy
from dataclasses import dataclass
from tqdm.notebook import tqdm
import numpy as np
import sys

In [ ]:
# A way to represent a state of the game
Nimply = namedtuple("Nimply", "row, num_objects")

In [ ]:
class Nim:
    def __init__(self, num_rows: int, k: int = sys.maxsize) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self._finalstage = False

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def final_stage(self) -> bool:
        return self._finalstage

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert num_objects <= self._k
        self._rows[row] -= num_objects
        self.__check_final_stage()

    def __check_final_stage(self) -> None:
        # Check if all the remaining rows are at 1 excepted one
        row_not_one = 0

        for row in self._rows:
            if row > 1:
                row_not_one += 1

        if row_not_one == 1:
            self._finalstage = True

In [ ]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, min(state._k, state._rows[row]))
    return Nimply(row, num_objects)

def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

def advanced_nim_sum(state: Nim) -> int:
    mex = [x % (state._k + 1) for x in state.rows]
    #print(mex)
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in mex])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        if raw._k < ply.num_objects:
            continue
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        # cooked["possible_moves"][ply] = nim_sum(tmp)
        cooked["possible_moves"][ply] = advanced_nim_sum(tmp)
        
    return cooked

def avoid_loosing_final_stage(raw: Nim, analysis: dict) -> dict:
    pruned = deepcopy(analysis)
    for ply in analysis["possible_moves"]:
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        if tmp._finalstage and advanced_nim_sum(tmp) != 1:
            pruned["possible_moves"].pop(ply)

    return pruned

def force_winning_final_stage(raw: Nim, analysis: dict) -> Nimply:
    for ply in analysis["possible_moves"]:
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        if tmp._finalstage and advanced_nim_sum(tmp) == 1:
            return ply

    return None

def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    if state._k != sys.maxsize:
        analysis = avoid_loosing_final_stage(state, analysis)
    if state._finalstage:
        spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 1]
    else:
        spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    
    if not spicy_moves:
        ply = pure_random(state)
    else:
        ply = random.choice(spicy_moves)
        
    return ply

In [ ]:
class CustomHashTable:
    def __init__(self):
        self.hash_table = {}

    def hash_function(self, key):
        hash_value = ord(key[0]) * 10000 + ord(key[1]) * 1000 + ord(key[2]) * 100 + ord(key[3]) * 10 + ord(key[4])
        return hash_value

    def insert(self, key, value):
        # index = self.hash_function(key)
        index = key
        if index not in self.hash_table:
            scores = []
            for _ in range(10):
                scores.append(random.randint(1, 10))
            self.hash_table[index] = scores
        self.hash_table[index] = value

    # if index not yet discovered, create it and initialize it with a random value between 1 and 10
    def get(self, key):
        # index = self.hash_function(key)
        index = key
        if index not in self.hash_table:
            scores = []
            for _ in range(10):
                scores.append(random.randint(1, 10))
            self.hash_table[index] = scores
        return self.hash_table[index]
    
    def get_keys(self):
        # here it should reconstruct the keys from numbers to strings
        return list(self.hash_table.keys())

In [ ]:
@dataclass
class Strategy:
    def __init__(self):
        self.hash_table = CustomHashTable()

In [ ]:
# Generates the initial population of strategies
def generate_initial_population(number_of_individuals: int) -> list[Strategy]:
    population = []
    for _ in range(number_of_individuals):
        population.append(
            Strategy())
    return population

In [ ]:
# generates a list with all the possible moves (of type Nimply) at the current state of the game
def possible_moves(state: Nim) -> list:
    possible_moves = []
    for ply in (Nimply(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)):
        try:
            tmp = deepcopy(state)
            tmp.nimming(ply)
            possible_moves.append(ply)
            # print("assertion ok")
        except AssertionError:
            print("assertion error")
    return possible_moves

In [ ]:
def describe_game(actual_state: Nim):
    state = deepcopy(actual_state)
    max_value = max(state.rows)

    if max_value > 3:
        return 4
    else:
        return max_value

In [ ]:
def describe_move(actual_state: Nim, move: Nimply):
    element_before_move = actual_state.rows[move.row]
    state = deepcopy(actual_state)
    state.nimming(move)
    new_elements = state.rows[move.row]

    move_description = ""
    if element_before_move > 3:
        move_description += "D"
    elif element_before_move == 3:
        move_description += "C"
    elif element_before_move == 2:
        move_description += "B"
    else:
        move_description += "A"

    if new_elements > 1:
        if new_elements % 2 == 0:
            move_description += "C"
        else:
            move_description += "D"
    else:
        move_description += chr(64 + 1 + new_elements)

    return move_description

In [ ]:
def generate_hash_key(actual_state: Nim):
    state = deepcopy(actual_state)
    key = ""

    # first character = [A, B, C, D] = category of the actual state
    state_category = max(state.rows)
    if state_category > 3:
        key += "D"
    else:
        key += chr(state_category + 64)

    # second character = [U, P, D, Z] = uno, pari, dispari or zero for rows of category A
    # third character = [U, P, D, Z] = uno, pari, dispari or zero for rows of category B
    # fourth character = [U, P, D, Z] = uno, pari, dispari or zero for rows of category C
    # fifth character = [U, P, D] = uno, pari, dispari for rows of category D
    a_category_rows = 0
    b_category_rows = 0
    c_category_rows = 0
    d_category_rows = 0
    for row in state.rows:
        if row > 3:
            d_category_rows += 1
        elif row == 3:
            c_category_rows += 1
        elif row == 2:
            b_category_rows += 1
        else:
            a_category_rows += 1

    if a_category_rows > 0:
        if a_category_rows == 1:
            key += "U"
        elif a_category_rows % 2 == 0:
            key += "P"
        else:
            key += "D"
    else:
        key += "Z"

    if b_category_rows > 0:
        if b_category_rows == 1:
            key += "U"
        elif b_category_rows % 2 == 0:
            key += "P"
        else:
            key += "D"
    else:
        key += "Z"

    if c_category_rows > 0:
        if c_category_rows == 1:
            key += "U"
        elif c_category_rows % 2 == 0:
            key += "P"
        else:
            key += "D"
    else:
        key += "Z"

    if d_category_rows > 0:
        if d_category_rows == 1:
            key += "U"
        elif d_category_rows % 2 == 0:
            key += "P"
        else:
            key += "D"
    else:
        key += "Z"

    return key  
    

In [ ]:
NUMBER_OF_FITNESS_GAMES = 200

In [ ]:
# this should let strategies play against each other and assign a score to each of them based on the number of wins
# OPTIMAL PLAYS FIST
def fitness_function(strategy: Strategy, k, num_games = NUMBER_OF_FITNESS_GAMES) -> int:
    score = 0
    for _ in range(num_games):
        # everytime the strategy wins it gets a point
        if nim_match(optimal, strategy, k, player = 1, fitness = True, ) == 1:
            score += 1
    return score

In [ ]:
def crossover(mother: Strategy, father: Strategy):
    child1 = Strategy()
    child2 = Strategy()

    set_keys_mother = set(mother.hash_table.get_keys())
    set_keys_father = set(father.hash_table.get_keys())
    set_keys_childrend = set_keys_mother.union(set_keys_father)

    for key in set_keys_childrend:
        if key in set_keys_mother and key in set_keys_father:
            child_1_scores = []
            child_2_scores = []

            mother_scores = mother.hash_table.get(key)
            father_scores = father.hash_table.get(key)

            i_love_mum = 1
            for i in range(10):
                if mother_scores[i] == 0 or father_scores[i] == 0:
                    child_1_scores.append(0)
                    child_2_scores.append(0)
                else:
                    if i_love_mum == 1:
                        child_1_scores.append(mother_scores[i])
                        child_2_scores.append(father_scores[i])
                        i_love_mum = 0
                    else:
                        child_1_scores.append(father_scores[i])
                        child_2_scores.append(mother_scores[i])
                        i_love_mum = 1
            child1.hash_table.insert(key, child_1_scores)
            child2.hash_table.insert(key, child_2_scores)
        elif key in set_keys_mother:
            child1.hash_table.insert(key, mother.hash_table.get(key))
            child2.hash_table.insert(key, mother.hash_table.get(key))
        else:
            child1.hash_table.insert(key, father.hash_table.get(key))
            child2.hash_table.insert(key, father.hash_table.get(key))

    return child1, child2

In [ ]:
def calculate_scores(population: list[Strategy], k) -> list:
    scores = []
    for strategy in tqdm(population):
        score = fitness_function(strategy, k)
        scores.append((strategy, score))
    
    # sort scores by fitnes descending
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

In [ ]:
# gaussian mutation
standard_deviation = 0.6
MUTATION_RATE = 0.3

In [ ]:
def mutate(strategy: Strategy) -> Strategy:
    mutated_strategy = Strategy()
    set_keys = set(strategy.hash_table.get_keys())
    for key in set_keys:
        scores = strategy.hash_table.get(key)
        for i in range(10):
            if scores[i] != 0:
                if random.random() < MUTATION_RATE:
                    mutation = random.gauss(0, standard_deviation)
                    # print("mutation: " + str(mutation))
                    mutated_score = scores[i] + mutation
                    mutated_score = max(0.01, min(10, mutated_score))
                    scores[i] = mutated_score
        mutated_strategy.hash_table.insert(key, scores)
    return mutated_strategy
    

In [ ]:
INITIAL_POPULATION = 256 # always divisible by 4
NUMBER_OF_GENERATIONS = 50

In [ ]:
# first try (version 7): only evolve for starting second
def evolve() -> Strategy:
    scores_player_0 = [] # scores of the strategies when optimal plays first
    not_growing_counter = 0
    for k in tqdm([3, 5, 7, 9, 10, 11]):
        print("-------------------- K = ", k, "-------------------- ")
        # generate random strategies (initial population)
        population = generate_initial_population(INITIAL_POPULATION)

        # calculate the fitness of each strategy
        scores = []
        scores = calculate_scores(population, k)

        best_score = scores[0][1]
        print("k: ", k, "best score: ", scores[0][1])

        # start the genetic algorithm
        for _ in tqdm(range(NUMBER_OF_GENERATIONS)):
            new_generation = []

            # select the best strategies
            # best_half = scores[:len(scores)//2]
            best_quarter = scores[:len(scores)//4]
            # best_ten = scores[:len(scores)//10]

            # while len(best_quarter) > 0:
            for _ in range((INITIAL_POPULATION - len(best_quarter)) // 2):
                # crossover
                # we pick randomly the mother and the father from the best quarter
                mother = random.choice(best_quarter)
                # best_quarter.pop(best_quarter.index(mother))
                father = random.choice(best_quarter)
                # best_quarter.pop(best_quarter.index(father))

                # generate two children
                child1, child2 = crossover(mother[0], father[0])

                # mutation

                child1 = mutate(child1)
                child2 = mutate(child2)

                new_generation.append(child1)
                new_generation.append(child2)
                # new_generation.append(mother[0])
                # new_generation.append(father[0])

            # calculate the fitness of each strategy
            scores = []
            scores = calculate_scores(new_generation, k)

            # to not recompute again the fitness of the best quarter
            scores += best_quarter
            scores.sort(key=lambda x: x[1], reverse=True)
            
            print("k: ", k, "best score: ", scores[0][1])

            if scores[0][1] <= best_score:
                not_growing_counter += 1
            else:
                best_score = scores[0][1]
                not_growing_counter = 0

            if not_growing_counter == 5:
                not_growing_counter = 0
                break

        scores_player_0.append(scores[0][0])

    return scores_player_0

In [ ]:
# corresponding index of move_scores in the hash_table of strategies
MOVES_DICTIONARY = {
    0: 'AA',
    1: 'BA',
    2: 'BB',
    3: 'CA',
    4: 'CB',
    5: 'CC',
    6: 'DA',
    7: 'DB',
    8: 'DC',
    9: 'DD',
}

In [ ]:
SET_OF_ALLOWED_MOVES = set()
for value in MOVES_DICTIONARY.values():
    SET_OF_ALLOWED_MOVES.add(value)

In [ ]:
# the function that returns the move to do 
def evolution_strategy(state: Nim, strategy: Strategy) -> Nimply:
    # generate the possible moves
    allowed_moves = possible_moves(state)

    # transform the list of allowed_moves into a list of move_types
    allowed_moves_types = []
    for move in allowed_moves:
        allowed_moves_types.append(describe_move(state, move))

    set_of_allowed_moves = set()
    for value in allowed_moves_types:
        set_of_allowed_moves.add(value)

    set_of_not_allowed_moves = SET_OF_ALLOWED_MOVES - set_of_allowed_moves

    # generate the hash_key of the state
    hash_key = generate_hash_key(state)
    # print("hash_key: ", hash_key)

    # retrieve the scores of the moves from the hash_table
    moves_scores = strategy.hash_table.get(hash_key)

    # for each index retrieved from calling MOVES_DICTIONARY with the index of set_of_not_allowed_moves we set the score to 0 of moves_scores
    for index in set_of_not_allowed_moves:
        for key, value in MOVES_DICTIONARY.items():
            if value == index:
                moves_scores[key] = 0

    # order moves_scores as a tuple of type (score, move_type) in descending order
    moves_scores = list(zip(moves_scores, allowed_moves_types))
    moves_scores.sort(key=lambda x: x[0], reverse=True)

    # starting from the first move, check if it is in the list of allowed moves, if yes, return it
    for move in moves_scores:
        if move[1] in allowed_moves_types:
            return allowed_moves[allowed_moves_types.index(move[1])]
        
    print("random move")
    return random.choice(allowed_moves)

In [ ]:
NIM_SIZE = 3

In [ ]:
# player = 0 -> strategy1 plays first
# player = 1 -> strategy2 plays first
def nim_match(strategy1, strategy2, nim_size = NIM_SIZE, player = 0, fitness = False, debug = False, winner = False):
    agents = (strategy1, strategy2)
    if debug:
        print("agent1: ", agents[0])
        print("agent2: ", agents[1])
    nim = Nim(nim_size)
    logging.info(f"init : {nim}")
    if debug:
        print(f"init : {nim}")
    while nim:
        if player == 0:
            selected_move = agents[player](nim)
        else:
            selected_move = evolution_strategy(nim, agents[player])
        logging.info(f"ply: player {player} plays {selected_move}")
        if debug:
            print(f"ply: player {player} plays {selected_move}")
        nim.nimming(selected_move)
        logging.info(f"status: {nim}")
        if debug:
            print(f"status: {nim}")
        player = 1 - player
    logging.info(f"status: Player {player} won!")
    if debug or winner:
        print(f"status: Player {player} won!")
    if fitness:
        return player

In [ ]:
best_strategy = evolve()

In [ ]:
matches = 1000
wins_first = 0
wins_second = 0

for _ in tqdm(range(matches)):
    wins_second += nim_match(optimal, best_strategy, nim_size = 7, debug = False, fitness = True)
    wins_first += nim_match(optimal, best_strategy, nim_size = 7, debug = False, player = 1, fitness = True)

print("matches played: ", matches)
print("---------------------------------")
print("wins playing first: ", wins_first)
print("percentage of wins playing fist: ", wins_first/matches * 100, "%")
print("---------------------------------")
print("wins playing second: ", wins_second)
print("percentage of wins playing second: ", wins_second/matches * 100, "%")
print("---------------------------------")

In [ ]:
import logging

logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, optimal)

nim = Nim(2)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")